In [20]:
import pandas as pd
import numpy as np
import json
import requests
import pytz
from datetime import timedelta

# Fetch JSON data from the URL
headers = {
    'User-Agent': 'Mozilla/5.0'
}
url = "https://query1.finance.yahoo.com/v7/finance/chart/spy?dataGranularity=1m&range=7d"
response = requests.get(url, headers=headers)
print(response.status_code)  # Should be 200 for a successful request
json_data = response.json()

# Extract relevant information from the JSON response
timestamp = json_data['chart']['result'][0]['timestamp']
close_prices = json_data['chart']['result'][0]['indicators']['quote'][0]['close']

# Create a DataFrame
df = pd.DataFrame({
    'timestamp': timestamp,
    'close': close_prices
})

# Drop rows where any of the indicators are NaN
df = df.dropna(subset=['timestamp', 'close']).reset_index(drop=True)
df = df.sort_values(by='timestamp')

# Check for NaN values in the DataFrame
if df.isna().any().any():
    print("Error: NaN values found in calculations.")
    problematic_rows = df[df.isna().any(axis=1)]
    print(problematic_rows)  # Print only relevant columns
    print('---')

# Calculate EMA
def calculate_ema(df, period):
    return df['close'].ewm(span=period, adjust=False).mean()

# Revised RSI calculation to uses EMA
def calculate_rsi(df, period=7):
    delta = df['close'].diff()

    # Separate gains and losses
    gains = delta.where(delta > 0, 0.0)
    losses = -delta.where(delta < 0, 0.0)

    # Calculate EMA multiplier
    alpha = 2 / (period + 1)

    # First averages (SMA)
    first_avg_gain = gains.iloc[:period].mean() if period <= len(gains) else gains.mean()
    first_avg_loss = losses.iloc[:period].mean() if period <= len(losses) else losses.mean()

    # Initialize series
    avg_gains = pd.Series(index=delta.index)
    avg_losses = pd.Series(index=delta.index)

    # Set first values
    avg_gains.iloc[period-1] = first_avg_gain
    avg_losses.iloc[period-1] = first_avg_loss

    # Calculate EMA
    for i in range(period, len(gains)):
        avg_gains.iloc[i] = alpha * gains.iloc[i] + (1 - alpha) * avg_gains.iloc[i-1]
        avg_losses.iloc[i] = alpha * losses.iloc[i] + (1 - alpha) * avg_losses.iloc[i-1]

    rs = avg_gains / np.where(avg_losses == 0, 1e-10, avg_losses)
    rsi = 100 - (100 / (1 + rs))

    # Ensure RSI stays within bounds
    rsi = np.clip(rsi, 0, 100)

    return avg_gains, avg_losses, rsi

one_min_df = df.copy()
print(len(one_min_df))

# Append EMA to DataFrame
one_min_df['EMA_5'] = calculate_ema(one_min_df, 5)
one_min_df['EMA_10'] = calculate_ema(one_min_df, 10)

# Calculate RSI and gain/loss
one_min_df['Gain'], one_min_df['Loss'], one_min_df['RSI'] = calculate_rsi(one_min_df)

# Drop the first XX rows
one_min_df = one_min_df.iloc[10:].reset_index(drop=True)

# Check for NaN values in the DataFrame
if one_min_df.isna().any().any():
    print("Error: NaN values found in calculations.")
    problematic_rows = one_min_df[one_min_df.isna().any(axis=1)]
    print(problematic_rows)  # Print only relevant columns

# Drop rows where any of the indicators are NaN
one_min_df = one_min_df.dropna(subset=['EMA_5', 'EMA_10', 'RSI']).reset_index(drop=True)

# Transform RSI to Int values based on specified rules
def transform_rsi_to_int(rsi):
    if rsi >= 69:
        return 1
    elif rsi <= 31:
        return -1
    else:
        return 0

one_min_df['RSI_INT'] = one_min_df['RSI'].apply(transform_rsi_to_int)

# Add EMA comparison columns
one_min_df['EMA_5_EMA_10'] = (one_min_df['EMA_5'] > one_min_df['EMA_10']).astype(int)

# Check for NaN values in the DataFrame
if one_min_df.isna().any().any():
    print("Error: NaN values found in calculations.")
    problematic_rows = one_min_df[one_min_df.isna().any(axis=1)]
    print(problematic_rows)  # Print only relevant columns

# Convert timestamp to datetime and set timezone to EST
one_min_df['datetime'] = pd.to_datetime(one_min_df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York')

# Define trading day start and end times
one_min_df['trading_day'] = one_min_df['datetime'].dt.floor('D')
one_min_df['market_open'] = one_min_df['trading_day'] + pd.Timedelta(hours=9, minutes=30)  # 9:30 AM
one_min_df['market_close'] = one_min_df['trading_day'] + pd.Timedelta(hours=16, minutes=0)  # 4:00 PM

# Create a mask to filter out the unwanted time periods
mask = ~((one_min_df['datetime'] < (one_min_df['market_open'] + pd.Timedelta(minutes=30))) |
          (one_min_df['datetime'] > (one_min_df['market_close'] - pd.Timedelta(minutes=5))))  # For 3 minute interval: minutes=60

one_min_df = one_min_df[mask].reset_index(drop=True)

one_min_df = one_min_df.drop(columns=['datetime', 'trading_day', 'market_open', 'market_close'])

json_string = one_min_df.to_dict(orient='records')

# Save the JSON string to a file
filename = f'/kaggle/working/spy_1min_regularhours_truncated_preprocessed.json'
with open(filename, 'w') as json_file:
    json.dump(json_string, json_file)

200
2731


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


In [21]:
three_min_df = df.copy()
three_min_df = three_min_df.iloc[2::3].reset_index(drop=True)
print(len(three_min_df))

# Append EMA to DataFrame
three_min_df['EMA_5'] = calculate_ema(three_min_df, 5)
three_min_df['EMA_10'] = calculate_ema(three_min_df, 10)

# Calculate RSI and gain/loss
three_min_df['Gain'], three_min_df['Loss'], three_min_df['RSI'] = calculate_rsi(three_min_df)

# Drop the first XX rows
three_min_df = three_min_df.iloc[10:].reset_index(drop=True)

# Check for NaN values in the DataFrame
if three_min_df.isna().any().any():
    print("Error: NaN values found in calculations.")
    problematic_rows = three_min_df[three_min_df.isna().any(axis=1)]
    print(problematic_rows)  # Print only relevant columns

# Drop rows where any of the indicators are NaN
three_min_df = three_min_df.dropna(subset=['EMA_5', 'EMA_10', 'RSI']).reset_index(drop=True)

three_min_df['RSI_INT'] = three_min_df['RSI'].apply(transform_rsi_to_int)

# Add EMA comparison columns
three_min_df['EMA_5_EMA_10'] = (three_min_df['EMA_5'] > three_min_df['EMA_10']).astype(int)

# Check for NaN values in the DataFrame
if three_min_df.isna().any().any():
    print("Error: NaN values found in calculations.")
    problematic_rows = three_min_df[three_min_df.isna().any(axis=1)]
    print(problematic_rows)  # Print only relevant columns

three_min_df = three_min_df.sort_values(by='timestamp')

# Convert timestamp to datetime and set timezone to EST
three_min_df['datetime'] = pd.to_datetime(three_min_df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York')

# Define trading day start and end times
three_min_df['trading_day'] = three_min_df['datetime'].dt.floor('D')
three_min_df['market_open'] = three_min_df['trading_day'] + pd.Timedelta(hours=9, minutes=30)  # 9:30 AM
three_min_df['market_close'] = three_min_df['trading_day'] + pd.Timedelta(hours=16, minutes=0)  # 4:00 PM

# Create a mask to filter out the unwanted time periods
mask = ~((three_min_df['datetime'] < (three_min_df['market_open'] + pd.Timedelta(minutes=30))) |
          (three_min_df['datetime'] > (three_min_df['market_close'] - pd.Timedelta(minutes=5))))  # For 3 minute interval: minutes=60

three_min_df = three_min_df[mask].reset_index(drop=True)

three_min_df = three_min_df.drop(columns=['datetime', 'trading_day', 'market_open', 'market_close'])

json_string = three_min_df.to_dict(orient='records')

# Save the JSON string to a file
filename = f'/kaggle/working/spy_3min_regularhours_truncated_preprocessed.json'
with open(filename, 'w') as json_file:
    json.dump(json_string, json_file)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


910


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


In [22]:
# '''
# # Load the JSON data
# one_min_data = pd.read_json('/kaggle/input/spy-regularhours/spy_1min_regularhours_truncated_preprocessed.json')
# three_min_data = pd.read_json('/kaggle/input/spy-regularhours/spy_3min_regularhours_truncated_preprocessed.json')
# '''

# spy_1min_regularhours_truncated_preprocessed.json / spy_3min_regularhours_truncated_preprocessed.json format '''
# [{
# "timestamp":"1738161000"
# "close":209.05
# "EMA_5_EMA_10":0
# "RSI_INT":-1
# },...]
# '''

# Create a Python program to backtest the futures algo trading performance:
# - Only trade 1 futures contract at the same time.
# - Go through two code snippets for 1-min and 3-min data separately by 1 time step:
#   - Create a variable 'direction' to store int 1 or 0 and initial value is -1.
#     - If the same EMA_5_EMA_10 value continues 10 times then assign it to the 'direction'.
#   - Create a variable 'signal' to store instruction string and initial value is 'null':
#     - If direction == 1 and RSI_INT == 1, then indicate 'short'.
#     - If direction == 1 and RSI_INT == -1, then indicate 'long'.
#     - If direction == 0 and RSI_INT == 1, then indicate 'short'.
#     - If direction == 0 and RSI_INT == -1, then indicate 'long'.
# - If there is no contract on hand and 1-min signal is changed, both 1-min and 3-min directions are the same, 
# then open position based on the 1-min singal and not in the opposite to its direction.
# But do nothing if 1-min singal is 'null' and either 1-min or 3-min direction is -1.
# - If there is a contract on hand and a signal is changed whether it is 1-min or 3-min, then close position.
# - If there is a contract on hand and a direction is changed whether it is 1-min or 3-min, then close position.
# - If there is a contract on hand and 1-min RSI reached middle level, then close position.
# - Must close position at the end of the day
# - Reset variables 'direction' and 'signal' when entering a new day
# - Store the accumulated gain USD
# - Store the accumulated charges (USD charges) for each open position and close position

In [58]:
import pandas as pd

# Load the JSON data
# one_min_data = pd.read_json('/kaggle/input/spy-regularhours/spy_1min_regularhours_truncated_preprocessed.json')
# three_min_data = pd.read_json('/kaggle/input/spy-regularhours/spy_3min_regularhours_truncated_preprocessed.json')

one_min_data = pd.read_json('/kaggle/working/spy_1min_regularhours_truncated_preprocessed.json')
three_min_data = pd.read_json('/kaggle/working/spy_3min_regularhours_truncated_preprocessed.json')

# Function to process data and backtest
def backtest(one_min_data, three_min_data):
    # Initialize variables
    charges = 1.87
    accumulated_gain = 0
    accumulated_charges = 0
    position = None  # Tracks the active position ('long' or 'short')
    
    one_min_direction = -1
    three_min_direction = -1
    last_one_min_direction = -1
    last_three_min_direction = -1
    one_min_direction_changed = False
    three_min_direction_changed = False
    one_min_signal = 'null'
    three_min_signal = 'null'
    last_one_min_signal = 'null'
    last_three_min_signal = 'null'
    one_min_signal_changed = False
    three_min_signal_changed = False
    last_ema_5_10_one_min = None
    last_ema_5_10_three_min = None
    one_min_ema_count = 0
    three_min_ema_count = 0
    
    timestep_to_form_a_trend = 10
    rsi_middle_upper = 51
    rsi_middle_lower = 49
    open_count = 0
    close_count = 0

    # Combine both datasets on timestamps for simultaneous processing
    one_min_data['timestamp'] = pd.to_datetime(one_min_data['timestamp'], unit='s')
    three_min_data['timestamp'] = pd.to_datetime(three_min_data['timestamp'], unit='s')
    one_min_data = one_min_data.sort_values('timestamp').reset_index(drop=True)
    three_min_data = three_min_data.sort_values('timestamp').reset_index(drop=True)

    # Backtesting logic
    for index, row in three_min_data.iterrows():
        timestamp = row['timestamp']
        
        three_min_close = row['close']
        three_min_ema_5_10 = row['EMA_5_EMA_10']
        three_min_rsi_int = row['RSI_INT']
        
        # Find the closest timestamp in 1-min data
        filtered_one_min_data = one_min_data[one_min_data['timestamp'] <= timestamp]
        if filtered_one_min_data.empty:
            # Skip processing if no matching 1-min timestamp is found
            print("Skip processing because 1-min timestamp is found")
            continue
        if filtered_one_min_data.iloc[-1]['timestamp'] != timestamp:
            print(f"1-min timestamp {filtered_one_min_data.iloc[-1]['timestamp']}. 3-min timestamp {timestamp}")
        
        one_min_row = filtered_one_min_data.iloc[-1]
        one_min_close = one_min_row['close']
        one_min_ema_5_10 = one_min_row['EMA_5_EMA_10']
        one_min_rsi_int = one_min_row['RSI_INT']
        one_min_rsi = one_min_row['RSI']

        # Reset variables at the start of a new day
        if index > 0 and timestamp.date() != three_min_data.iloc[index - 1]['timestamp'].date():
            yesterday = three_min_data.iloc[index - 1]['timestamp'].date()
            yesterday_last_timestamp = three_min_data.iloc[index - 1]['timestamp'].time()
            yesterday_last_one_min_close = three_min_data.iloc[index - 1]['close']
            print(f'{timestamp.date()}, {yesterday}')
            if position is not None:
                # Close the position at the end of the day
                print(f'End of the day. Close {position}, Close price {one_min_close}, Entry price {entry_price}, yesterday_last_timestamp {yesterday_last_timestamp}')
                accumulated_gain += (yesterday_last_one_min_close - entry_price) if position == 'long' else entry_price - yesterday_last_one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            last_one_min_direction = -1
            last_three_min_direction = -1
            one_min_direction = -1
            three_min_direction = -1
            one_min_direction_changed = False
            three_min_direction_changed = False
            last_one_min_signal = 'null'
            last_three_min_signal = 'null'
            one_min_signal = 'null'
            three_min_signal = 'null'
            one_min_signal_changed = False
            three_min_signal_changed = False
            last_ema_5_10_one_min = None
            last_ema_5_10_three_min = None
            one_min_ema_count = 0
            three_min_ema_count = 0

        # Update one-minute direction
        if one_min_ema_5_10 == last_ema_5_10_one_min:
            one_min_ema_count += 1
        else:
            one_min_ema_count = 1
        last_ema_5_10_one_min = one_min_ema_5_10
        if one_min_ema_count == timestep_to_form_a_trend:
            last_one_min_direction = one_min_direction
            one_min_direction = one_min_ema_5_10
            one_min_direction_changed = True
        else:
            one_min_direction_changed = False
        
        # Update one-minute signal
        if one_min_direction == 1 and one_min_rsi_int == 1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'short'
            one_min_signal_changed = True
        elif one_min_direction == 1 and one_min_rsi_int == -1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'long'
            one_min_signal_changed = True
        elif one_min_direction == 0 and one_min_rsi_int == 1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'short'
            one_min_signal_changed = True
        elif one_min_direction == 0 and one_min_rsi_int == -1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'long'
            one_min_signal_changed = True
        else:
            one_min_signal_changed = False

        # Update three-minute direction
        if three_min_ema_5_10 == last_ema_5_10_three_min:
            three_min_ema_count += 1
        else:
            three_min_ema_count = 1
        last_ema_5_10_three_min = three_min_ema_5_10
        if three_min_ema_count == timestep_to_form_a_trend:
            last_three_min_direction = three_min_direction
            three_min_direction = three_min_ema_5_10
            three_min_direction_changed = True
        else:
            three_min_direction_changed = False

        # Update three-minute signal
        if three_min_direction == 1 and three_min_rsi_int == 1:
            last_three_min_signal = three_min_signal
            three_min_signal = 'short'
            three_min_signal_changed = True
        elif three_min_direction == 1 and three_min_rsi_int == -1:
            last_three_min_signal = three_min_signal
            three_min_signal = 'long'
            three_min_signal_changed = True
        elif three_min_direction == 0 and three_min_rsi_int == 1:
            last_three_min_signal = three_min_signal
            three_min_signal = 'short'
            three_min_signal_changed = True
        elif three_min_direction == 0 and three_min_rsi_int == -1:
            last_three_min_signal = three_min_signal
            three_min_signal = 'long'
            three_min_signal_changed = True
        else:
            three_min_signal_changed = False

        # Check for trading actions
        if position is None:
            if one_min_signal != 'null' and one_min_direction == three_min_direction:
                if one_min_direction != -1 and three_min_direction != -1:
                    if one_min_signal_changed:
                        if (one_min_direction == 1 and one_min_signal == 'long') or (one_min_direction == 0 and one_min_signal == 'short'):
                            if three_min_signal_changed and three_min_signal != one_min_signal and three_min_signal != 'null':
                                continue
                            else:
                                # Open a new position
                                position = one_min_signal
                                entry_price = one_min_close
                                accumulated_charges += charges  # Opening charge
                                open_count += 1
                                print(f'Open {position}, {timestamp.time()}, Entry price {entry_price}')
        else:
            if one_min_signal_changed and one_min_signal != last_one_min_signal:
                print(f'1 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}')
                # Close the position
                accumulated_gain += (one_min_close - entry_price) if position == 'long' else entry_price - one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            # elif three_min_signal_changed and three_min_signal != last_three_min_signal:
            #     print(f'2 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}')
            #     # Close the position
            #     accumulated_gain += (one_min_close - entry_price) if position == 'long' else entry_price - one_min_close
            #     accumulated_charges += charges  # Closing charge
            #     position = None
            #     close_count += 1
            elif (one_min_direction_changed and one_min_direction != last_one_min_direction) or (three_min_direction_changed and three_min_direction != last_three_min_direction):
                print(f'3 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}')
                # Close the position if direction changes
                accumulated_gain += (one_min_close - entry_price) if position == 'long' else entry_price - one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            elif position == 'long' and one_min_rsi >= rsi_middle_lower:
                print(f'4 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}. Reached RSI middle level {one_min_rsi}')
                # Close the position if direction changes
                accumulated_gain += one_min_close - entry_price
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            elif position == 'short' and one_min_rsi <= rsi_middle_upper:
                print(f'5 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}. Reached RSI middle level {one_min_rsi}')
                # Close the position if direction changes
                accumulated_gain += entry_price - one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1

    # Close any open position at the end of the backtest
    if position is not None:
        print(f'End of the backtest. Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}')
        accumulated_gain += (one_min_close - entry_price) if position == 'long' else entry_price - one_min_close
        accumulated_charges += charges  # Closing charge
        close_count += 1

    return accumulated_gain, accumulated_charges, open_count, close_count

# Run the backtest
gain, charges, open_count, close_count = backtest(one_min_data, three_min_data)
print(f"Accumulated Gain: ${gain:.2f}")
print(f"Accumulated Charges: ${charges:.2f}")
print(f"Open count: {open_count}")
print(f"Close count: {close_count}")

Open long, 15:50:00, Entry price 602.0303955078125
4 Close long, 15:56:00, Close price 602.3099975585938, Entry price 602.0303955078125. Reached RSI middle level 58.60939108234055
Open long, 16:05:00, Entry price 601.9912109375
4 Close long, 16:20:00, Close price 601.0499877929688, Entry price 601.9912109375. Reached RSI middle level 52.45457065402983
Open short, 16:38:00, Entry price 601.489990234375
5 Close short, 16:47:00, Close price 601.8499755859375, Entry price 601.489990234375. Reached RSI middle level 50.8344585726435
Open short, 16:50:00, Entry price 602.1649780273438
3 Close short, 16:59:00, Close price 603.3900146484375, Entry price 602.1649780273438
Open long, 17:32:00, Entry price 603.4299926757812
4 Close long, 17:35:00, Close price 603.7625122070312, Entry price 603.4299926757812. Reached RSI middle level 58.67524938727167
Open long, 17:41:00, Entry price 603.385009765625
1 Close long, 17:47:00, Close price 603.8200073242188, Entry price 603.385009765625
Open long, 18:1

In [62]:
import pandas as pd

# Load the JSON data
one_min_data = pd.read_json('/kaggle/input/spy-regularhours/spy_1min_regularhours_truncated_preprocessed.json')

# one_min_data = pd.read_json('/kaggle/working/spy_1min_regularhours_truncated_preprocessed.json')

# Function to process data and backtest
def backtest(one_min_data):
    # Initialize variables
    charges = 1.87
    accumulated_gain = 0
    accumulated_charges = 0
    position = None  # Tracks the active position ('long' or 'short')
    
    one_min_direction = -1
    last_one_min_direction = -1
    one_min_direction_changed = False
    one_min_signal = 'null'
    last_one_min_signal = 'null'
    one_min_signal_changed = False
    last_ema_5_10_one_min = None
    one_min_ema_count = 0
    
    timestep_to_form_a_trend = 10
    rsi_middle_upper = 51
    rsi_middle_lower = 49
    open_count = 0
    close_count = 0

    # Combine both datasets on timestamps for simultaneous processing
    one_min_data['timestamp'] = pd.to_datetime(one_min_data['timestamp'], unit='s')
    one_min_data = one_min_data.sort_values('timestamp').reset_index(drop=True)

    # Backtesting logic
    for index, row in one_min_data.iterrows():
        timestamp = row['timestamp']
        
        one_min_close = row['close']
        one_min_ema_5_10 = row['EMA_5_EMA_10']
        one_min_rsi_int = row['RSI_INT']
        one_min_rsi = row['RSI']

        # Reset variables at the start of a new day
        if index > 0 and timestamp.date() != one_min_data.iloc[index - 1]['timestamp'].date():
            yesterday = one_min_data.iloc[index - 1]['timestamp'].date()
            yesterday_last_timestamp = one_min_data.iloc[index - 1]['timestamp'].time()
            yesterday_last_one_min_close = one_min_data.iloc[index - 1]['close']
            print(f'{timestamp.date()}, {yesterday}')
            if position is not None:
                # Close the position at the end of the day
                print(f'End of the day. Close {position}, Close price {one_min_close}, Entry price {entry_price}, yesterday_last_timestamp {yesterday_last_timestamp}')
                accumulated_gain += (yesterday_last_one_min_close - entry_price) if position == 'long' else entry_price - yesterday_last_one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            last_one_min_direction = -1
            one_min_direction = -1
            one_min_direction_changed = False
            last_one_min_signal = 'null'
            one_min_signal = 'null'
            one_min_signal_changed = False
            last_ema_5_10_one_min = None
            one_min_ema_count = 0

        # Update one-minute direction
        if one_min_ema_5_10 == last_ema_5_10_one_min:
            one_min_ema_count += 1
        else:
            one_min_ema_count = 1
        last_ema_5_10_one_min = one_min_ema_5_10
        if one_min_ema_count == timestep_to_form_a_trend:
            last_one_min_direction = one_min_direction
            one_min_direction = one_min_ema_5_10
            one_min_direction_changed = True
        else:
            one_min_direction_changed = False
        
        # Update one-minute signal
        if one_min_direction == 1 and one_min_rsi_int == 1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'short'
            one_min_signal_changed = True
        elif one_min_direction == 1 and one_min_rsi_int == -1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'long'
            one_min_signal_changed = True
        elif one_min_direction == 0 and one_min_rsi_int == 1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'short'
            one_min_signal_changed = True
        elif one_min_direction == 0 and one_min_rsi_int == -1:
            last_one_min_signal = one_min_signal
            one_min_signal = 'long'
            one_min_signal_changed = True
        else:
            one_min_signal_changed = False

        # Check for trading actions
        if position is None:
            if one_min_signal != 'null' and one_min_direction != -1 and one_min_signal_changed:
                if (one_min_direction == 1 and one_min_signal == 'long') or (one_min_direction == 0 and one_min_signal == 'short'):
                    # Open a new position
                    position = one_min_signal
                    entry_price = one_min_close
                    accumulated_charges += charges  # Opening charge
                    open_count += 1
                    print(f'Open {position}, {timestamp.time()}, Entry price {entry_price}')
        else:
            if one_min_signal_changed and one_min_signal != last_one_min_signal:
                print(f'1 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}')
                # Close the position
                accumulated_gain += (one_min_close - entry_price) if position == 'long' else entry_price - one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            elif one_min_direction_changed and one_min_direction != last_one_min_direction:
                print(f'2 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}')
                # Close the position if direction changes
                accumulated_gain += (one_min_close - entry_price) if position == 'long' else entry_price - one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            elif position == 'long' and one_min_rsi >= rsi_middle_lower:
                print(f'3 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}. Reached RSI middle level {one_min_rsi}')
                # Close the position if direction changes
                accumulated_gain += one_min_close - entry_price
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1
            elif position == 'short' and one_min_rsi <= rsi_middle_upper:
                print(f'4 Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}. Reached RSI middle level {one_min_rsi}')
                # Close the position if direction changes
                accumulated_gain += entry_price - one_min_close
                accumulated_charges += charges  # Closing charge
                position = None
                close_count += 1

    # Close any open position at the end of the backtest
    if position is not None:
        print(f'End of the backtest. Close {position}, {timestamp.time()}, Close price {one_min_close}, Entry price {entry_price}')
        accumulated_gain += (one_min_close - entry_price) if position == 'long' else entry_price - one_min_close
        accumulated_charges += charges  # Closing charge
        close_count += 1

    return accumulated_gain, accumulated_charges, open_count, close_count

# Run the backtest
gain, charges, open_count, close_count = backtest(one_min_data)
print(f"Accumulated Gain: ${gain:.2f}")
print(f"Accumulated Charges: ${charges:.2f}")
print(f"Open count: {open_count}")
print(f"Close count: {close_count}")

Open short, 15:23:00, Entry price 603.7550048828125
1 Close short, 15:26:00, Close price 603.2999877929688, Entry price 603.7550048828125
Open short, 15:35:00, Entry price 603.3699951171875
4 Close short, 15:42:00, Close price 603.4400024414062, Entry price 603.3699951171875. Reached RSI middle level 48.409514116644246
Open long, 15:50:00, Entry price 603.1500244140625
2 Close long, 15:59:00, Close price 601.4998779296875, Entry price 603.1500244140625
Open short, 16:19:00, Entry price 601.5197143554688
2 Close short, 16:30:00, Close price 602.969970703125, Entry price 601.5197143554688
Open long, 17:00:00, Entry price 603.75
3 Close long, 17:02:00, Close price 603.89501953125, Entry price 603.75. Reached RSI middle level 50.56585524655257
Open long, 17:06:00, Entry price 603.4600219726562
3 Close long, 17:08:00, Close price 603.7999267578125, Entry price 603.4600219726562. Reached RSI middle level 55.66668261740971
Open long, 17:12:00, Entry price 603.4199829101562
2 Close long, 17:13